# ENTREGABLE 4

# INSTRUCCIONES

Utilizar el archivo CSV (`dataset_banco_clean.csv`) con 45189 filas y 17 columnas y aplicar las técnicas de normalización del entregable 3.

In [1]:
# imports
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd

In [2]:
ruta = "./dataset_banco_clean.csv"
data = pd.read_csv(ruta)
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143.0,yes,no,unknown,5,may,261.0,1,-1.0,0,unknown,no
1,44,technician,single,secondary,no,29.0,yes,no,unknown,5,may,151.0,1,-1.0,0,unknown,no
2,33,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5,may,76.0,1,-1.0,0,unknown,no
3,47,blue-collar,married,unknown,no,1506.0,yes,no,unknown,5,may,92.0,1,-1.0,0,unknown,no
4,33,unknown,single,unknown,no,1.0,no,no,unknown,5,may,198.0,1,-1.0,0,unknown,no


# Objetivo

Generar un model de clasificación capaz de predecir la clase de flor en función de las carácterísticas del dataset

* Aplicar las técnicas oportunas de procesamiento de datos

* Generar split de los datos

* Valorar diferentes modelos de clasificación

* Comparación entre modelos

* Ensemble

* Métricas

* Conclusiones finales

first of all, we are going to transform the categorical values into numbers

In [3]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
numerical_columns = ['balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

for column in categorical_columns:
    data[column] = encoder.fit_transform(data[column])

data['y'] = encoder.fit_transform(data['y'])

In [4]:
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,4,1,2,0,2143.0,1,0,2,5,8,261.0,1,-1.0,0,3,0
1,44,9,2,1,0,29.0,1,0,2,5,8,151.0,1,-1.0,0,3,0
2,33,2,1,1,0,2.0,1,1,2,5,8,76.0,1,-1.0,0,3,0
3,47,1,1,3,0,1506.0,1,0,2,5,8,92.0,1,-1.0,0,3,0
4,33,11,2,3,0,1.0,0,0,2,5,8,198.0,1,-1.0,0,3,0


now we are going to normalice the data

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

data_2 = data.copy()
data_2[numerical_columns] = scaler.fit_transform(data[numerical_columns])

In [6]:
data_2.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,4,1,2,0,0.195954,1,0,2,-1.298557,8,0.011211,-0.569436,-0.411352,-0.300937,3,0
1,44,9,2,1,0,-0.342737,1,0,2,-1.298557,8,-0.416228,-0.569436,-0.411352,-0.300937,3,0
2,33,2,1,1,0,-0.349617,1,1,2,-1.298557,8,-0.707664,-0.569436,-0.411352,-0.300937,3,0
3,47,1,1,3,0,0.033633,1,0,2,-1.298557,8,-0.645491,-0.569436,-0.411352,-0.300937,3,0
4,33,11,2,3,0,-0.349872,0,0,2,-1.298557,8,-0.233595,-0.569436,-0.411352,-0.300937,3,0


now the split

In [7]:
from sklearn.model_selection import train_test_split

X = data_2.drop('y', axis=1)
y = data_2['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'{X_train.shape}, {y_train.shape}')
print(f'{X_test.shape}, {y_test.shape}')

(36151, 16), (36151,)
(9038, 16), (9038,)


we are going to choose the knn model

In [8]:
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

knn_classifier = KNeighborsClassifier()

knn_classifier.fit(X_train, y_train)

y_pred_knn = knn_classifier.predict(X_test)

accuracy_knn = accuracy_score(y_test, y_pred_knn)
print("kNN accuracy:", accuracy_knn)

kNN accuracy: 0.8888028324850631


now we are going to use the gradientBoosting model whitch ensamble other models

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)

gb_model.fit(X_train, y_train)

accuracy_gb = gb_model.score(X_test, y_test)

print("Gradient Boosting accuracy:", accuracy_gb)


Gradient Boosting accuracy: 0.908165523345873


we can see that it is a little bit better than knn model

lets see the confusion matrix of the models

In [10]:
from sklearn.metrics import classification_report, confusion_matrix

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)

    print("Matriz de confusión:")
    print(confusion_matrix(y_test, y_pred))


print("\nMétricas para kNN:")
evaluate_model(knn_classifier, X_test, y_test)

print("\nMétricas para Gradient Boosting:")
evaluate_model(gb_model, X_test, y_test)



Métricas para kNN:
Matriz de confusión:
[[7781  197]
 [ 808  252]]

Métricas para Gradient Boosting:
Matriz de confusión:
[[7792  186]
 [ 644  416]]


we can see that the gradient boosting confusion matrix is much better than the knn confusion matrix

# Conclusiones

we have seen that enembing models is a good practice, its not only about the accuracy but also on how the clasifier clasifies